# 基于MindSpore构造卷积层

本实验主要专注于卷积层的原理和构造，使用MindSpore构造卷积层。

## 1、实验目的

- 掌握卷积层的原理和特征。
- 掌握如何使用MindSpore构造卷积层。

## 2、卷积层原理介绍

卷积层的主要作用是提取图像中的特征，浅层卷积层用来提取边缘、纹理等特征，深层卷积层用来提取更加抽象的特征，整合在一起构成了丰富的图像特征。

卷积层由一组和输入层有着相同通道数的卷积核构成，将卷积核在输入层上按照指定步长滑动，每到一个位置计算卷积核权重和对应输入特征的乘积，所有乘积求和作为输出特征图的一个特征值，单个卷积核在输入特征上滑动一遍就得到了单通道的输出特征，一组卷积核全部进行上述过程就得到了包含若干通道的输出特征图。这种每次滑动到一个局部位置的计算方式能够在非常大的程度上减少参数量，同时一个物体无论在图像中的什么位置，这一组卷积核都能够提取其特征，具有平移不变性。

以单通道为例，卷积的简易过程如图所示，从左到右三部分依次为输入图像、卷积核（过滤器）、输出特征，输入图像尺寸为5×5，卷积核尺寸为3×3，输出特征图尺寸为3×3。卷积核中心滑动到输入图像中某一像素的位置，则重叠的部分分别计算乘积，最后求和作为当前位置的输出特征值。
<img src="./Figures/fig001.jpg" style="zoom:70%" />

图中的卷积计算过程如下：

第一行卷积的结果为：1×(-1) + 0×(-1) + 0×(-1) = -1 

第二行卷积的结果为：1×(-1) + 0× 1 + 1×(-1) = -2

第三行卷积的结果为：1×(-1) + 0×(-1) + 1×(-1) = -2

然后把三行的结果相加等于-5。

卷积层输出特征图尺寸计算方式：输入特征图尺寸为$w_i×h_i×c_i$（分别为宽、高、通道数），卷积核的尺寸为$k_i×k_i$（k表示卷积核的边长），输出通道数为$n_{i+1}$，卷积步长为$s$，输入特征图周围填充0特征值的行数为$p_0$，输出特征图的尺寸为$w_{i+1}×h_{i+1}×c_{i+1}$，则输出特征图的尺寸为：
$$w_{i+1}=\frac{w_i-k_i+2p_0}{s}+1$$
$$h_{i+1}=\frac{h_i-k_i+2p_0}{s}+1$$
$$c_{i+1}=n_{i+1}$$

当卷积结束之后，可以发现图像变小了，原来是5×5的，现在变成了3×3的，这不是想要的结果，因此进行零填充，因为填充的是二维图像，所以在图像四周都要填成0。填充完之后再开始做卷积运算，中间部分已得出，需要计算新填充的部分，用同样的方式计算完所有的位置，二维卷积的零填充如图所示：
<img src="./Figures/fig002.jpg" style="zoom:70%" />
原来的图像是5×5的，零填充之后，得到的也是5×5，大小保持不变。
根据上面的公式，我们可以计算输出特征图的尺寸。输入特征图尺寸为5×5×1，卷积核的尺寸为3×3，卷积步长$s$为1，输入特征图周围填充0特征值的行数$p_0$为1，根据公式得到输出特征图的尺寸为5×5×1。
$$w_{i+1}=\frac{5-3+2}{1}+1=5$$
$$h_{i+1}=\frac{5-3+2}{1}+1=5$$
$$c_{i+1}=n_{i+1}=1$$

## 3、实验环境

在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

## 4、数据处理

### 4.1 数据准备

对输入Tensor计算二维卷积。该Tensor的常见shape为$(N,C_{in},H_{in},W_{in})$，其中N为batch size，$C_{in}$为空间维度，$H_{in}$,$W_{in}$分别为特征层的高度和宽度。现给出Tensor的shape为(1,120,1024,640)计算二维卷积。

### 4.2 数据加载

导入需要的Python库，调用mindspore.nn.Conv2d()，它是对输入Tensor计算二维卷积，其中has_bias设置为默认值'False'，weight_init设置为'normal'，然后用给出的Tensor进行数据加载，Tensor的shape为(1,120,1024,640)，然后输出。

In [2]:
from mindspore import Tensor
import mindspore.nn as nn
import numpy as np
from mindspore import dtype as mstype

# 给出Tensor的shape为(1,120,1024,640)计算二维卷积。
net = nn.Conv2d(120, 240, 4, has_bias=False, weight_init='normal')
# 输入
x = Tensor(np.ones([1, 120, 1024, 640]), mstype.float32)

## 5、模型构建

**导入Python库&模块**

在使用前，导入需要的Python库。

In [3]:
# 导入科学计算库
import numpy as np
# 导入神经网络模块
import mindspore.nn as nn
# 被初始化的Tensor的数据类型
from mindspore import dtype as mstype
# 常见操作
from mindspore.nn import Conv2d
# 用于初始化Tensor的Tensor
from mindspore import Tensor
# MindSpore中神经网络的基本构成单元
from mindspore.nn import Cell

## 6、模型预测

给定Tensor的shape为(1,120,1024,640)，然后调用Conv2d函数来计算二维卷积，最后输出结果。

In [4]:
net = Conv2d(120, 240, 3, has_bias=False, weight_init='normal')
# 输入
x = Tensor(np.ones([1, 120, 1024, 640]), mstype.float32)
# 输出
output = net(x)
print(output[:,6])
print(output.shape)

[[[-0.06043273 -0.06498916 -0.06498916 ... -0.06498916 -0.06498916
    0.07462145]
  [-0.19988076 -0.13575162 -0.13575162 ... -0.13575162 -0.13575162
   -0.02684124]
  [-0.19988076 -0.13575162 -0.13575162 ... -0.13575162 -0.13575162
   -0.02684124]
  ...
  [-0.19988076 -0.13575162 -0.13575162 ... -0.13575162 -0.13575162
   -0.02684124]
  [-0.19988076 -0.13575162 -0.13575162 ... -0.13575162 -0.13575162
   -0.02684124]
  [-0.25204647 -0.24795136 -0.24795136 ... -0.24795136 -0.24795136
   -0.16285488]]]
(1, 240, 1024, 640)


## 7、使用numpy实现二维卷积

**导入Python库&模块**

In [5]:
# 导入科学计算库
import numpy as np

**用numpy实现二维卷积运算**

In [6]:
class my_Conv2d:
    # 初始化
    def __init__(self, 
                 input_channel,    # 输入Tensor的空间维度
                 output_channel,   # 输出Tensor的空间维度
                 kernel_size,      # 指定二维卷积核的高度和宽度
                 stride=1,         # 二维卷积核的移动步长
                 padding=1,        # 输入的高度和宽度方向上填充的数量
                 bias=False,       # 二维卷积层是否添加偏置参数，默认值：False
                 dilation=1):      # 二维卷积核膨胀尺寸
        self.input_channel = input_channel
        self.output_channel = output_channel
        self.stride = stride
        self.padding = padding
        self.dilation = (dilation, dilation) if isinstance(dilation, int) else dilation
        self.kernel_size = (kernel_size, kernel_size) if isinstance(kernel_size, int) else kernel_size
        self.weight = np.random.randn(output_channel, input_channel, self.kernel_size[0], self.kernel_size[1])
        self.bias = None
        if bias:
            self.bias = np.random.randn(output_channel)

    def __call__(self, inputs):
        return self.infer(inputs)

    def infer(self, inputs):
        # 根据参数，算出输出的shape
        # print(inputs.shape)
        batch_size, input_channel, height, width = inputs.shape
        output_h = (height + 2 * self.padding - self.dilation[0] * (self.kernel_size[0] - 1) - 1) // self.stride + 1
        output_w = (width + 2 * self.padding - self.dilation[1] * (self.kernel_size[1] - 1) - 1) // self.stride + 1
        outputs = np.zeros([batch_size, self.output_channel, output_h, output_w])
        
        # 计算padding之后的inputs_array
        inputs_padding = np.zeros([batch_size, input_channel, height + 2 * self.padding, width + 2 * self.padding])
        inputs_padding[:, :, self.padding: self.padding + height, self.padding:self.padding + width] = inputs

        # 如果有dilation，根据dilation之后的shape往kernel中插入0（注意，原self.weight不变）
        dilation_shape = self.dilation[0] * (self.kernel_size[0] - 1) + 1, self.dilation[1] * (self.kernel_size[1] - 1) + 1
        kernel = np.zeros((self.output_channel, input_channel, dilation_shape[0], dilation_shape[1]))
        # print(kernel)
        if self.dilation[0] > 1:
            for i in range(self.kernel_size[0]):
                for j in range(self.kernel_size[1]):
                    kernel[:, :, self.dilation[0] * i, self.dilation[1] * j] = self.weight[:, :, i, j]
        else:
            kernel = self.weight
        # print(output_h,output_w)
        
        # 开始前向计算
        for h in range(output_h):
            for w in range(output_w):
                input_ = inputs_padding[
                         :,
                         :,
                         h * self.stride:h * self.stride + dilation_shape[0],
                         w * self.stride:w * self.stride + dilation_shape[1]
                         ]
                # input_ shape : batch_size, output_channel, input_channel, dilation_shape[0], dilation_shape[1]
                input_ = np.repeat(input_[:, np.newaxis, :, :, :], self.output_channel, axis=1)
                # print(input_)
                # kernel_ shape: batch_size, output_channel, input_channel, dilation_shape[0], dilation_shape[1]
                kernel_ = np.repeat(kernel[np.newaxis, :, :, :, :], batch_size, axis=0)
                # print(kernel_.shape, input_.shape)
                # output shape: batch_size, output_channel
                output = input_ * kernel_
                output = np.sum(output, axis=(-1, -2, -3))
                outputs[:, :, h, w] = output
                # print(h,w)

        if self.bias is not None:
            bias_ = np.tile(self.bias.reshape(-1, 1), (1, output_h * output_w)).\
                reshape(self.output_channel, output_h, output_w)
            outputs += bias_
        return outputs

## 8、实验对比

基于Mindspore构造的二维卷积和使用numpy实现的二维卷积进行对比，本次实验中，构造二维卷积定义变量时，设置了相同的参数，如：二维卷积核的移动步长都设置为1，二维卷积核的膨胀尺寸都设置为1，输入的高度和宽度方向上填充的数量都设置为0等等。在进行模型预测时，基于Mindspore构造的二维卷积规定Tensor的shape为(1,120,1024,640)，同时使用numpy实现的二维卷积也采用同样的输入，然后计算二维卷积，输出结果。由实验结果可知，两者结果相同，并且两个实验都输出了相同的shape(1,240,1024,640)。

In [ ]:
net = Conv2d(120, 240, 3, has_bias=False, weight_init='normal')
# mindspore
input_1 = Tensor(np.ones([1, 120, 1024, 640]), mstype.float32)
output_1 = net(input_1)
print('mindspore result shape:\n',output_1.shape)
print('mindspore result:\n',output_1[:,6])
# numpy
input_2 = np.ones([1, 120, 1024, 640])
my_net = my_Conv2d(120, 240, 3, stride=1, dilation=1)
my_net.weight = net.weight.numpy()
output_2 = my_net(input_2)
print('numpy result shape:\n',output_2.shape)
print('numpy result:\n',output_2[:,6])
print(output.shape)

mindspore result shape:
 (1, 240, 1024, 640)
mindspore result:
 [[[-0.24489996 -0.2758379  -0.2758379  ... -0.2758379  -0.2758379
   -0.16903809]
  [-0.2283787  -0.26414382 -0.26414382 ... -0.26414382 -0.26414382
   -0.29966903]
  [-0.2283787  -0.26414382 -0.26414382 ... -0.26414382 -0.26414382
   -0.29966903]
  ...
  [-0.2283787  -0.26414382 -0.26414382 ... -0.26414382 -0.26414382
   -0.29966903]
  [-0.2283787  -0.26414382 -0.26414382 ... -0.26414382 -0.26414382
   -0.29966903]
  [-0.11650491 -0.28843373 -0.28843373 ... -0.28843373 -0.28843373
   -0.36278778]]]
numpy result shape:
 (1, 240, 1024, 640)
numpy result:
 [[[-0.24489996 -0.2758379  -0.2758379  ... -0.2758379  -0.2758379
   -0.16903808]
  [-0.22837871 -0.26414383 -0.26414383 ... -0.26414383 -0.26414383
   -0.29966905]
  [-0.22837871 -0.26414383 -0.26414383 ... -0.26414383 -0.26414383
   -0.29966905]
  ...
  [-0.22837871 -0.26414383 -0.26414383 ... -0.26414383 -0.26414383
   -0.29966905]
  [-0.22837871 -0.26414383 -0.26414383